In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from backup.module.preprocess import drop_columns

fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'AppleGothic'}) # 폰트 설정
plt.rc('font', family='AppleGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
# sys.path를 수정하여 모듈 경로를 설정
import sys
import os
import pandas as pd

# 프로젝트 루트 경로 추가
project_root = os.path.abspath("..")
sys.path.append(project_root)
# 모든 컬럼 출력 설정
# pd.set_option('display.max_columns', None)
# 모든 행 출력 설정
# pd.set_option('display.max_rows', None)

import module

In [4]:
df, df_test_y = module.file_load.load_current_version()

ValueError: too many values to unpack (expected 2)

In [4]:
df['is_test'].value_counts()      # train과 test data가 하나로 합쳐진 것을 확인할 수 있습니다.

is_test
0    1097155
1       5607
Name: count, dtype: int64

In [5]:
# 결측치가 많거나 필요 없는 컬럼을 정리한다
drop_columns = [
    '본번','부번',
       '해제사유발생일', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '단지신청일'
]

# 전처리

In [6]:
module.preprocess.컬럼_정리(df)
df_select = module.preprocess.drop_columns(df, drop_columns)
module.preprocess.번지_본번_부번_결측채우기(df_select)
df_select = module.preprocess.remove_outliers_iqr(df_select, '전용면적')
df_select['좌표X'].fillna(0, inplace=True)
df_select['좌표Y'].fillna(0, inplace=True)

21.71
122.91


In [7]:
df_select['좌표Y'].value_counts()

좌표Y
0.000000     789096
37.512752      2969
37.480002      2816
37.497741      2495
37.642724      2428
              ...  
37.493087         1
37.580320         1
37.559515         1
37.496296         1
37.561735         1
Name: count, Length: 725, dtype: int64

# 파생변수

In [8]:
module.preprocess.시_군_구_add(df_select)
module.preprocess.강남_add(df_select)
module.preprocess.신축_add(df_select)
# module.preprocess.계약_건축년도_차이(df_select)

# Model Training

## 범주형 변수 Encoding

In [9]:
model_data = df_select.copy()

In [10]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1016001 entries, 0 to 5606
Data columns (total 19 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   번지       1016001 non-null  object 
 1   아파트명     1016001 non-null  object 
 2   전용면적     1016001 non-null  float64
 3   계약년월     1016001 non-null  int64  
 4   계약일      1016001 non-null  int64  
 5   층        1016001 non-null  int64  
 6   건축년도     1016001 non-null  int64  
 7   도로명      1016001 non-null  object 
 8   등기신청일자   1016001 non-null  object 
 9   거래유형     1016001 non-null  object 
 10  중개사소재지   1016001 non-null  object 
 11  좌표X      1016001 non-null  float64
 12  좌표Y      1016001 non-null  float64
 13  target   1010394 non-null  float64
 14  is_test  1016001 non-null  int64  
 15  시        1016001 non-null  object 
 16  구        1016001 non-null  object 
 17  동        1016001 non-null  object 
 18  강남여부     1016001 non-null  int64  
dtypes: float64(4), int64(6), object(9)
memory usage: 1

In [11]:
model_data.isnull().sum()

번지            0
아파트명          0
전용면적          0
계약년월          0
계약일           0
층             0
건축년도          0
도로명           0
등기신청일자        0
거래유형          0
중개사소재지        0
좌표X           0
좌표Y           0
target     5607
is_test       0
시             0
구             0
동             0
강남여부          0
dtype: int64

In [12]:
dt_train, dt_test = module.preprocess.split_train_test(model_data)
categorical_columns_v2 = module.preprocess.범주형_filter(dt_train)
module.preprocess.범주형_encoding(dt_train, dt_test, categorical_columns_v2)
print(dt_train.shape, dt_test.shape)

  0%|          | 0/9 [00:00<?, ?it/s]

번지


 22%|██▏       | 2/9 [00:00<00:01,  4.53it/s]

아파트명
도로명


 56%|█████▌    | 5/9 [00:00<00:00,  6.05it/s]

...

구


 89%|████████▉ | 8/9 [00:01<00:00,  5.30it/s]

동


100%|██████████| 9/9 [00:01<00:00,  5.12it/s]

(1010394, 18) (5607, 18)


## Model Training


# 훈련데이터 분할

In [13]:
X_train, X_val, y_train, y_val = module.preprocess.holdout_split_X_y(dt_train)

In [14]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((808315, 17), (202079, 17), (808315,), (202079,))

In [15]:
X_train.head(1)

,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,등기신청일자,거래유형,중개사소재지,좌표X,좌표Y,시,구,동,강남여부
11242,768,3892,114.85,201804,5,5,2004,4658,0,0,0,0.0,0.0,0,2,110,0


In [16]:
X_train.isnull().sum()

번지        0
아파트명      0
전용면적      0
계약년월      0
계약일       0
층         0
건축년도      0
도로명       0
등기신청일자    0
거래유형      0
중개사소재지    0
좌표X       0
좌표Y       0
시         0
구         0
동         0
강남여부      0
dtype: int64

In [17]:
# RandomForestRegressor 를 이용해 회귀 모델을 적합시키겠습니다.
model = RandomForestRegressor(n_estimators=5, criterion='squared_error', random_state=1, n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_val)

In [18]:
df_select.isnull().sum()

번지            0
아파트명          0
전용면적          0
계약년월          0
계약일           0
층             0
건축년도          0
도로명           0
등기신청일자        0
거래유형          0
중개사소재지        0
좌표X           0
좌표Y           0
target     5607
is_test       0
시             0
구             0
동             0
강남여부          0
dtype: int64

In [19]:
print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, pred))}')

RMSE test: 5381.468744996257


In [21]:
X_test = dt_test.drop(['target'], axis=1)

real_test_pred = model.predict(X_test)
print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(df_test_y, real_test_pred))}')

RMSE test: 42291.20835216755
